In [37]:
import numpy as np     #importing necessary libraries
import pandas as pd
import statistics
import time

#libraries of classifiers are imported at the time of use

data = pd.read_csv('leukemia.csv')      #reading leukemia data file.

data.gene[data.gene == 'ALL'] = 1     #changing class label
data.gene[data.gene == 'AML'] = 2

df = data.iloc[2:,:].copy()         #removing two unuseful rows from data-frame


df.head()

,gene,AFFX-BioC-5_at,hum_alu_at,AFFX-DapX-M_at,AFFX-LysX-5_at,AFFX-HUMISGF3A/M97935_MA_at,AFFX-HUMISGF3A/M97935_MB_at,AFFX-HUMISGF3A/M97935_3_at,AFFX-HUMRGE/M10098_5_at,AFFX-HUMRGE/M10098_M_at,...,M93143_at,U29175_at,U48730_at,U58516_at,X06956_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at
2,1,88,15091,311,21,-13,215,797,14538,9738,...,384,1582,185,511,389,793,329,36,191,-37
3,1,283,11038,134,-21,-219,116,433,615,115,...,231,624,169,837,442,782,295,11,76,-14
4,1,309,16692,378,67,104,476,1474,5669,3272,...,720,753,315,1199,168,1138,777,41,228,-41
5,1,12,15763,268,43,-148,155,415,4850,2293,...,307,743,240,835,174,627,170,-50,126,-91
6,1,168,18128,118,-8,-55,122,483,1284,2731,...,178,626,156,649,504,250,314,14,56,-25


In [38]:
x= df.values.copy()               #converting data-frame into numpy arrray
x

array([[1, '88', '15091', ..., '36', '191', '-37'],
       [1, '283', '11038', ..., '11', '76', '-14'],
       [1, '309', '16692', ..., '41', '228', '-41'],
       ...,
       [2, '146', '15689', ..., '-2', '210', '16'],
       [2, '431', '41570', ..., '0', '284', '-73'],
       [2, '9', '39538', ..., '20', '379', '-60']], dtype=object)

In [39]:
a,b = x.shape             #size(row,column) of 2-d array stored in two variables

n1 =0;
n2 = 0;

for i in range(a):        #calculating size of each class lable
    if x[i][0] == 1:
        n1+=1;
    else:
        n2+=1;


sample1 = []
sample2= []
tval=[]

#below is main loop for calculating t-value of each column

f_start_time = time.time()

for j in range(1,b):      #for each column(feature)
    for i in range(a):

        if x[i][0]== 1:           #dividing feature values into two samples corresponding to class levels  1 and 2.
            sample1.append(x[i][j])
        else:
            sample2.append(x[i][j])



    sample1 = list(map(float,sample1))       #converting sample values from string to float
    sample2 = list(map(float,sample2))



    s1bar = statistics.mean(sample1)          #calculating mean of each sample
    s2bar = statistics.mean(sample2)

    s1var = statistics.variance(sample1)      #calculating variance of each sample
    s2var = statistics.variance(sample2)



    t = abs(s1bar - s2bar)/np.sqrt(s1var/n1+s2var/n2)        #calculating t value by comparing mean and variance using given formula

    tval.append(t)       #appending t-value of each sample to a list

    sample1.clear()      #clearing sample list  as it will be reused for each column(feature)
    sample2.clear()

tval

print('Time taken by t-test is :'+str(time.time() - f_start_time)+' seconds')

Time taken by t-test is :5.700271368026733 seconds


In [40]:
sort_index=[]

sort_index = sorted(range(len(tval)), key=lambda k: tval[k])   #sorting t-value in ascending order and storing corresponding index


new_index = [p+1 for p in sort_index]    #increasing sorted index by 1 since  we have to fetch it from dataframe for corresponding columns

new_index.reverse()     # it will sort the index in descending order
print(len(new_index))

# Note: columns having more t- values are more important hence we will select less than 20% of features from starting of the calculated new_index list

# here we are selecting 12 % of features

a = int(0.12*len(new_index))
print(a)
new_index = new_index[:a].copy()   #selecting important features



new_index1 = [0] + new_index      #adding 0 to the column index so that we can fetch dataframe including labels


df1 = df.iloc[:,new_index1].copy()   #fetching dataframe with important features

#print('new data frame after  important feature  selection through t-test :')
print(df1)    #this is the new dataframe containing most important features using t-test

print(df1.shape) #size of new dataframe


5147
617
   gene X59417_at M92287_at M31523_at U05259_rna1_at X95735_at  \
2     1      3016      4778      1320           9326       298   
3     1      3424      2700       898            895       307   
4     1      7724      4926       597            628       309   
5     1      3821      5403      1644           5314       693   
6     1      5216      3440      1322           5354       713   
7     1      4194      3179       787            282       247   
8     1      5528      3978       946           5146        52   
9     1      2073      3293      1917           4048      -215   
10    1      7782      7324      1440            423      -223   
11    1      5204      2676       442            585       492   
12    1      6541      6134       617            147       -25   
13    1       846      2056       474           2504       408   
14    1      7743     10669      1969           5075       724   
15    1      1756      1912       686            155       360   
1

In [41]:
df1.gene[df1.gene == 1] = 'ALL'
df1.gene[df1.gene == 2] = 'AML'



X = df1.iloc[:,1:].values.copy()  #selecting features for training
Y= df1.iloc[:,0].values.copy()    #selecting labels

#splitting dataset into training and test set(20%)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20,random_state=1)


#scaling features for better classification

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#training knn classifier with k value as 5


from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

#prediction based on test set

y_pred = classifier.predict(X_test)

#calculating accuracy ,confusion matrix and f-score for k-nn

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

#training with svm linear kernal

from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)

#predicton of svm
y_pred = clf.predict(X_test)

#all accuracy reports, f-score and confusion matrices

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[10  0]
 [ 1  4]]
              precision    recall  f1-score   support

         ALL       0.91      1.00      0.95        10
         AML       1.00      0.80      0.89         5

    accuracy                           0.93        15
   macro avg       0.95      0.90      0.92        15
weighted avg       0.94      0.93      0.93        15

[[10  0]
 [ 1  4]]
              precision    recall  f1-score   support

         ALL       0.91      1.00      0.95        10
         AML       1.00      0.80      0.89         5

    accuracy                           0.93        15
   macro avg       0.95      0.90      0.92        15
weighted avg       0.94      0.93      0.93        15

